In [ ]:
import tensorflow as tf
import keras
from keras import layers
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
!pip install tensorflow-model-optimization
from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks
import tensorflow_model_optimization as tfmot
from tensorflow_model_optimization.sparsity import keras as sparsity
from tensorflow.keras.models import load_model
from google.colab import files
import shutil
import gc
from keras.datasets import cifar100
from google.colab import drive
drive.mount('/content/drive')

np.random.seed(42)
tf.random.set_seed(42)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
num_classes = 100
input_shape = (32, 32, 3)

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



x_train shape: (50000, 32, 32, 3, 1)
50000 train samples
10000 test samples


In [ ]:
model_name = 'LeNet'
dataset = 'CIFAR100'

---

# Distillation

In [ ]:
def train_custom(model, train_data, train_labels, test_data, test_labels):
    initailsoftmax = model.student.predict(test_data)
    model.student.save_weights(init_weights)
    model.student.save(init_model)
    initaildf = pd.DataFrame(initailsoftmax)
    filename = filepath + '0_softmax.csv'
    initaildf.to_csv(filename,index=False)
    epoch = 1
    while epoch < 26:
        print(f"Epoch {epoch}:")

        # Training on one epoch
        model.fit(train_data, train_labels, epochs=1, batch_size=64, verbose=1)

        # Evaluate on the test dataset
        results = model.student.predict(test_data)
        softmax_df = pd.DataFrame(results)
        filename = filepath+str(epoch)+'_softmax.csv'
        softmax_df.to_csv(filename,index=False)
        loss,accuracy = model.student.evaluate(test_data,test_labels)
        print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

        if epoch ==25:
          model.student.save_weights(final_weights)
          model.student.save(final_model)
        epoch += 1


In [ ]:
class Distiller(tf.keras.Model):
    def __init__(self, student, teacher):
        super().__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha,
        temperature,
    ):
        super().compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)

            distillation_loss = (
                self.distillation_loss_fn(
                    tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                    tf.nn.softmax(student_predictions / self.temperature, axis=1),
                )
                * self.temperature**2
            )

            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )

        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)
        print(y_prediction)
        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

In [ ]:
teacher_model_path = '/content/base/LeNetCIFAR100_final.tf'
techer_model = load_model(teacher_model_path)

# Student 0.1 Alpha

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model_path = '/content/base/LeNetCIFAR100_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=3,
)
filepath = '/content/student_same/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 1s 2ms/step
Epoch 1:
313/313 [==============================] - 1s 3ms/step - loss: 3.7319 - accuracy: 0.1485
Test Loss: 3.731912612915039, Test Accuracy: 0.148499995470047
Epoch 2:
313/313 [==============================] - 1s 3ms/step - loss: 3.4092 - accuracy: 0.1990
Test Loss: 3.40924334526062, Test Accuracy: 0.19900000095367432
Epoch 3:
313/313 [==============================] - 1s 3ms/step - loss: 3.2158 - accuracy: 0.2357
Test Loss: 3.2158336639404297, Test Accuracy: 0.23569999635219574
Epoch 4:
313/313 [==============================] - 1s 3ms/step - loss: 3.0912 - accuracy: 0.2571
Test Loss: 3.091240644454956, Test Accuracy: 0.257099986076355
Epoch 5:
313/313 [==============================] - 1s 4ms/step - loss: 2.9959 - accuracy: 0.2778
Test Loss: 2.9959261417388916, Test Accuracy: 0.2777999937534332
Epoch 6:
313/313 [==============================] - 1s 4ms/step - loss: 2.9273 - accuracy: 0.2908
Test Loss: 2.9273061752319336, Test 

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same", 'zip', "student_same")
colab_link = "/content/student_same.zip"
gdrive_link = "/content/drive/MyDrive/LeNet_D_C100"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/LeNet_D_C100/student_same.zip'

# Alpha 0.1 (2)

In [ ]:
np.random.seed(24)
tf.random.set_seed(24)

model_path = '/content/base/LeNetCIFAR100_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=3,
)
filepath = '/content/student_same_2/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same_2/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 3s 3ms/step
Epoch 1:
313/313 [==============================] - 1s 3ms/step - loss: 3.7363 - accuracy: 0.1371
Test Loss: 3.7363405227661133, Test Accuracy: 0.1370999962091446
Epoch 2:
313/313 [==============================] - 1s 4ms/step - loss: 3.4068 - accuracy: 0.2013
Test Loss: 3.4068360328674316, Test Accuracy: 0.2012999951839447
Epoch 3:
313/313 [==============================] - 1s 3ms/step - loss: 3.2235 - accuracy: 0.2305
Test Loss: 3.223472833633423, Test Accuracy: 0.2304999977350235
Epoch 4:
313/313 [==============================] - 1s 3ms/step - loss: 3.0960 - accuracy: 0.2602
Test Loss: 3.096013307571411, Test Accuracy: 0.26019999384880066
Epoch 5:
313/313 [==============================] - 1s 3ms/step - loss: 3.0006 - accuracy: 0.2734
Test Loss: 3.0006492137908936, Test Accuracy: 0.2734000086784363
Epoch 6:
313/313 [==============================] - 1s 3ms/step - loss: 2.9296 - accuracy: 0.2903
Test Loss: 2.929598331451416, Tes

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_2", 'zip', "student_same_2")
colab_link = "/content/student_same_2.zip"
gdrive_link = "/content/drive/MyDrive/LeNet_D_C100"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/LeNet_D_C100/student_same_2.zip'

# Alpha 0.1 (3)

In [ ]:
np.random.seed(2)
tf.random.set_seed(2)

model_path = '/content/base/LeNetCIFAR100_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=3,
)
filepath = '/content/student_same_3/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same_3/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 1s 2ms/step
Epoch 1:
313/313 [==============================] - 1s 3ms/step - loss: 3.7159 - accuracy: 0.1511
Test Loss: 3.715893030166626, Test Accuracy: 0.15109999477863312
Epoch 2:
313/313 [==============================] - 1s 3ms/step - loss: 3.4264 - accuracy: 0.1957
Test Loss: 3.426447629928589, Test Accuracy: 0.195700004696846
Epoch 3:
313/313 [==============================] - 1s 4ms/step - loss: 3.2294 - accuracy: 0.2292
Test Loss: 3.2293665409088135, Test Accuracy: 0.22920000553131104
Epoch 4:
313/313 [==============================] - 1s 4ms/step - loss: 3.0844 - accuracy: 0.2590
Test Loss: 3.084412097930908, Test Accuracy: 0.2590000033378601
Epoch 5:
313/313 [==============================] - 1s 3ms/step - loss: 2.9810 - accuracy: 0.2789
Test Loss: 2.981017827987671, Test Accuracy: 0.27889999747276306
Epoch 6:
313/313 [==============================] - 1s 3ms/step - loss: 2.9050 - accuracy: 0.2956
Test Loss: 2.904978036880493, Test

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_3", 'zip', "student_same_3")
colab_link = "/content/student_same_3.zip"
gdrive_link = "/content/drive/MyDrive/LeNet_D_C100"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/LeNet_D_C100/student_same_3.zip'

# Student same Alpha 0.5

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)
model_path = '/content/base/LeNetCIFAR100_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.5,
    temperature=3,
)
filepath = '/content/student_same_50/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same_50/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 1s 2ms/step
Epoch 1:
313/313 [==============================] - 1s 3ms/step - loss: 3.7358 - accuracy: 0.1489
Test Loss: 3.7357523441314697, Test Accuracy: 0.14890000224113464
Epoch 2:
313/313 [==============================] - 1s 3ms/step - loss: 3.4341 - accuracy: 0.1920
Test Loss: 3.4341418743133545, Test Accuracy: 0.19200000166893005
Epoch 3:
313/313 [==============================] - 1s 3ms/step - loss: 3.2397 - accuracy: 0.2305
Test Loss: 3.2397067546844482, Test Accuracy: 0.2304999977350235
Epoch 4:
313/313 [==============================] - 1s 3ms/step - loss: 3.1147 - accuracy: 0.2535
Test Loss: 3.114743947982788, Test Accuracy: 0.2535000145435333
Epoch 5:
313/313 [==============================] - 1s 3ms/step - loss: 3.0155 - accuracy: 0.2716
Test Loss: 3.0154805183410645, Test Accuracy: 0.27160000801086426
Epoch 6:
313/313 [==============================] - 1s 4ms/step - loss: 2.9369 - accuracy: 0.2853
Test Loss: 2.936919689178467, 

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_50", 'zip', "student_same_50")
colab_link = "/content/student_same_50.zip"
gdrive_link = "/content/drive/MyDrive/LeNet_D_C100"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/LeNet_D_C100/student_same_50.zip'

# Student 0.5 (2)

In [ ]:
np.random.seed(24)
tf.random.set_seed(24)
model_path = '/content/base/LeNetCIFAR100_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.5,
    temperature=3,
)
filepath = '/content/student_same_50_2/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same_50_2/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 1s 2ms/step
Epoch 1:
313/313 [==============================] - 1s 3ms/step - loss: 3.7332 - accuracy: 0.1461
Test Loss: 3.7331743240356445, Test Accuracy: 0.1460999995470047
Epoch 2:
313/313 [==============================] - 1s 3ms/step - loss: 3.4248 - accuracy: 0.1958
Test Loss: 3.424762487411499, Test Accuracy: 0.19580000638961792
Epoch 3:
313/313 [==============================] - 1s 5ms/step - loss: 3.2219 - accuracy: 0.2313
Test Loss: 3.22192120552063, Test Accuracy: 0.2312999963760376
Epoch 4:
313/313 [==============================] - 1s 3ms/step - loss: 3.0781 - accuracy: 0.2604
Test Loss: 3.078061103820801, Test Accuracy: 0.2603999972343445
Epoch 5:
313/313 [==============================] - 1s 3ms/step - loss: 2.9751 - accuracy: 0.2793
Test Loss: 2.9750680923461914, Test Accuracy: 0.2793000042438507
Epoch 6:
313/313 [==============================] - 1s 3ms/step - loss: 2.9005 - accuracy: 0.2938
Test Loss: 2.900493621826172, Test 

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_50_2", 'zip', "student_same_50_2")
colab_link = "/content/student_same_50_2.zip"
gdrive_link = "/content/drive/MyDrive/LeNet_D_C100"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/LeNet_D_C100/student_same_50_2.zip'

# student 0.5 (3)

In [ ]:
np.random.seed(2)
tf.random.set_seed(2)
model_path = '/content/base/LeNetCIFAR100_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.5,
    temperature=3,
)
filepath = '/content/student_same_50_3/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same_50_3/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 1s 3ms/step
Epoch 1:
313/313 [==============================] - 1s 3ms/step - loss: 3.7127 - accuracy: 0.1510
Test Loss: 3.712696075439453, Test Accuracy: 0.1509999930858612
Epoch 2:
313/313 [==============================] - 1s 3ms/step - loss: 3.3971 - accuracy: 0.2008
Test Loss: 3.3971126079559326, Test Accuracy: 0.20080000162124634
Epoch 3:
313/313 [==============================] - 1s 3ms/step - loss: 3.1935 - accuracy: 0.2375
Test Loss: 3.193453073501587, Test Accuracy: 0.23749999701976776
Epoch 4:
313/313 [==============================] - 1s 4ms/step - loss: 3.0555 - accuracy: 0.2651
Test Loss: 3.0554966926574707, Test Accuracy: 0.26510000228881836
Epoch 5:
313/313 [==============================] - 1s 3ms/step - loss: 2.9583 - accuracy: 0.2827
Test Loss: 2.958258867263794, Test Accuracy: 0.2827000021934509
Epoch 6:
313/313 [==============================] - 1s 3ms/step - loss: 2.8848 - accuracy: 0.3002
Test Loss: 2.8847618103027344, T

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_50_3", 'zip', "student_same_50_3")
colab_link = "/content/student_same_50_3.zip"
gdrive_link = "/content/drive/MyDrive/LeNet_D_C100"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/LeNet_D_C100/student_same_50_3.zip'

# Student same Alpha 0.9

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)
model_path = '/content/base/LeNetCIFAR100_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)

distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=3,
)

filepath = '/content/student_same_90/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'

!mkdir '/content/student_same_90/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 1s 2ms/step
Epoch 1:
313/313 [==============================] - 1s 3ms/step - loss: 3.7210 - accuracy: 0.1498
Test Loss: 3.7210450172424316, Test Accuracy: 0.14980000257492065
Epoch 2:
313/313 [==============================] - 1s 3ms/step - loss: 3.4118 - accuracy: 0.1967
Test Loss: 3.411792516708374, Test Accuracy: 0.19670000672340393
Epoch 3:
313/313 [==============================] - 1s 3ms/step - loss: 3.2062 - accuracy: 0.2360
Test Loss: 3.20623779296875, Test Accuracy: 0.23600000143051147
Epoch 4:
313/313 [==============================] - 1s 3ms/step - loss: 3.0769 - accuracy: 0.2606
Test Loss: 3.0768651962280273, Test Accuracy: 0.2606000006198883
Epoch 5:
313/313 [==============================] - 1s 4ms/step - loss: 2.9869 - accuracy: 0.2793
Test Loss: 2.9869091510772705, Test Accuracy: 0.2793000042438507
Epoch 6:
313/313 [==============================] - 1s 3ms/step - loss: 2.9185 - accuracy: 0.2940
Test Loss: 2.918541431427002, Te

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_90", 'zip', "student_same_90")
colab_link = "/content/student_same_90.zip"
gdrive_link = "/content/drive/MyDrive/LeNet_D_C100"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/LeNet_D_C100/student_same_90.zip'

# studnet 0.9 (2)

In [ ]:
np.random.seed(24)
tf.random.set_seed(24)
model_path = '/content/base/LeNetCIFAR100_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)

distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=3,
)

filepath = '/content/student_same_90_2/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'

!mkdir '/content/student_same_90_2/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 1s 2ms/step
Epoch 1:
313/313 [==============================] - 1s 3ms/step - loss: 3.7387 - accuracy: 0.1428
Test Loss: 3.7386858463287354, Test Accuracy: 0.1428000032901764
Epoch 2:
313/313 [==============================] - 1s 3ms/step - loss: 3.4420 - accuracy: 0.1916
Test Loss: 3.4419801235198975, Test Accuracy: 0.1915999948978424
Epoch 3:
313/313 [==============================] - 1s 4ms/step - loss: 3.2437 - accuracy: 0.2254
Test Loss: 3.243666887283325, Test Accuracy: 0.22540000081062317
Epoch 4:
313/313 [==============================] - 1s 3ms/step - loss: 3.1034 - accuracy: 0.2530
Test Loss: 3.103449821472168, Test Accuracy: 0.2529999911785126
Epoch 5:
313/313 [==============================] - 1s 3ms/step - loss: 2.9979 - accuracy: 0.2721
Test Loss: 2.997920036315918, Test Accuracy: 0.2721000015735626
Epoch 6:
313/313 [==============================] - 1s 3ms/step - loss: 2.9215 - accuracy: 0.2906
Test Loss: 2.9214892387390137, Tes

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_90_2", 'zip', "student_same_90_2")
colab_link = "/content/student_same_90_2.zip"
gdrive_link = "/content/drive/MyDrive/LeNet_D_C100"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/LeNet_D_C100/student_same_90_2.zip'

# student 0.9 (3)

In [ ]:
np.random.seed(2)
tf.random.set_seed(2)
model_path = '/content/base/LeNetCIFAR100_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)

distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=3,
)

filepath = '/content/student_same_90_3/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'

!mkdir '/content/student_same_90_3/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 1s 2ms/step
Epoch 1:
313/313 [==============================] - 1s 3ms/step - loss: 3.7382 - accuracy: 0.1473
Test Loss: 3.738187551498413, Test Accuracy: 0.14730000495910645
Epoch 2:
313/313 [==============================] - 1s 3ms/step - loss: 3.4348 - accuracy: 0.1933
Test Loss: 3.434767484664917, Test Accuracy: 0.19329999387264252
Epoch 3:
313/313 [==============================] - 1s 3ms/step - loss: 3.2403 - accuracy: 0.2258
Test Loss: 3.240344762802124, Test Accuracy: 0.22579999268054962
Epoch 4:
313/313 [==============================] - 1s 3ms/step - loss: 3.0972 - accuracy: 0.2556
Test Loss: 3.0971591472625732, Test Accuracy: 0.2556000053882599
Epoch 5:
313/313 [==============================] - 1s 3ms/step - loss: 2.9912 - accuracy: 0.2775
Test Loss: 2.9911563396453857, Test Accuracy: 0.2775000035762787
Epoch 6:
313/313 [==============================] - 1s 3ms/step - loss: 2.9140 - accuracy: 0.2940
Test Loss: 2.9139809608459473, T

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_90_3", 'zip', "student_same_90_3")
colab_link = "/content/student_same_90_3.zip"
gdrive_link = "/content/drive/MyDrive/LeNet_D_C100"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/LeNet_D_C100/student_same_90_3.zip'